In [18]:
from sklearn.base import BaseEstimator, ClassifierMixin
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## $\texttt{myDecisionTreeClassifier}$

El modelo soporta dos criterios de partición:
- $\textbf{ID3}$: utiliza la entropía y la ganancia de información.
- $\textbf{Gini}$: utiliza el índice de Gini y la reducción de impureza.

El constructor del modelo:

```python
MyDecisionTreeClassifier(criterio="gini", 
                         max_depth=None, 
                         min_samples_split=2)
```

Define los siguientes hiperparámetros:

- `criterio`: puede ser \texttt{"id3"} o \texttt{"gini"}.  
    - Si es $\texttt{"id3"}$, el modelo usa entropía y ganancia de información
        como en ID3.
    - Si es $\texttt{"gini"}$, utiliza el índice Gini característico de CART.

- `max_depth`: profundización máxima del árbol, 
    implementando pre-poda.
    
- `min_samples_split`: tamaño mínimo de un nodo para poder dividirse.
    


En ID3 original no existe poda, pero el temario recomienda técnicas 
de pre-poda para evitar sobreajuste.

---
### Criterios

#### Entropía (ID3)

La función:

```python
def entropy(self, y):
    ...
```

implementa exactamente la definición teórica:

$$
H(C) = -\sum_{i=1}^{K} p(C_i)\,\log_2 p(C_i),
$$
Esto quiere decir:
- $H(C)=0$ si el nodo es puro,
- $H(C)$ es máxima cuando las clases están uniformemente distribuidas.

#### Gini

La función:

```python
def gini(self, y):
    ...
```

implementa la fórmula teórica del índice Gini:

$$
Gini(T) = 1 - \sum_{i=1}^{K} p_i^2,
$$

El índice Gini es cero cuando todas las instancias pertenecen a la misma clase.

---

### Ganancia de Información / Reducción de Impureza

La función:

```python
information_gain(self, y, y_left, y_right, impurity_func)
```
calcula la mejora de impureza producida al dividir un nodo en dos subconjuntos.

Sea $T$ el nodo padre y $T_L$, $T_R$ los nodos hijos.
$$
\begin{align}
\text{Gain} &= 
\text{Impureza}(T) \;-\;
\left(
    \frac{|T_L|}{|T|} \text{Impureza}(T_L)
    +
    \frac{|T_R|}{|T|} \text{Impureza}(T_R)
\right)
\end{align}
$$

- Con entropía, esto coincide exactamente con la $\textbf{Ganancia de Información}$:
    $$
    I(C,X) = H(C) - H(C \mid X)
    $$
- Con Gini, coincide con la $\textbf{reducción de impureza de CART}$.

---
### Selección del Mejor Punto de Corte

La función:

```python
best_split(self, X, y)
```

evalúa todas las características y todos los posibles umbrales para encontrar la partición que maximiza la ganancia.


1. Ordenar los valores del atributo.
    \item Calcular puntos medios entre valores consecutivos:
    $$
        t_j = \frac{v_j + v_{j+1}}{2}.
    $$
2. Evaluar los splits binarios:
    $$
        X_i < t_j \qquad \text{y} \qquad X_i \ge t_j.
    $$
3.  Seleccionar el umbral que maximiza la reducción de impureza.



La función devuelve:
- el índice de la mejor característica;
- el mejor umbral encontrado.

Si ningún split mejora la impureza, se devuelve $\texttt{None}$ y el nodo se convertirá en hoja.

---

### Construcción Recursiva del Árbol

La función:

```python
build_tree(self, X, y, depth)
```

implementa el algoritmo recursivo para ID3 y CART:

1. Crear un nodo con la clase mayoritaria.
2. Comprobar condiciones de parada:
    - nodo puro ,
    - profundidad máxima alcanzada (pre-poda),
    - tamaño mínimo insuficiente (pre-poda).
3. Buscar el mejor split mediante `best_split`.
    \item Si existe un split válido:
    - asignar atributo y umbral al nodo,
    - dividir los datos en hijos izquierdo y derecho,
    - llamar recursivamente al método para construir subárboles.

--- 
### Predicción

La predicción se realiza mediante un recorrido desde la raíz hasta una hoja:

- En cada nodo interno se evalúa la condición:
    $$
        x[\text{feature}] \le \text{threshold}.
    $$
- Se sigue la rama correspondiente.
- Al alcanzar un nodo hoja, se devuelve la clase almacenada.


In [16]:
class myDecisionTreeClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, criterio="gini", max_depth=None, min_samples_split=2):
        self.criterion = criterio
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree_ = None # Almacenará el árbol de decisión entrenado

    # Criterios

    # Gini
    def gini(self, y):
        _, counts = np.unique(y, return_counts=True)
        p = counts / counts.sum()
        return 1.0 - np.sum(p**2)

    # ID3
    def entropy(self, y):
        _, counts = np.unique(y, return_counts=True)
        p = counts / counts.sum()
        return -np.sum(p * np.log2(p + 1e-12)) # Se añade un valor pequeño para evitar el log(0)

    def information_gain(self, y, y_left, y_right, impurity_func):
        # Seleccionar el atributo que produzca la mayor reducción de la entropía de la clase después de la partición
        impurity_parent = impurity_func(y)
        n = len(y)
        n_left, n_right = len(y_left), len(y_right)
        
        if n_left == 0 or n_right == 0:
            return 0.0

        impurity_children = (n_left / n) * impurity_func(y_left) + \
                            (n_right / n) * impurity_func(y_right)

        gain = impurity_parent - impurity_children
        return gain

    def best_split(self, X, y):
        n_samples, n_features = X.shape
        
        if n_samples < self.min_samples_split: # No dividir si no hay más del mínimo de muestras
            return None, None
        
        # Elige la función de impureza
        if self.criterion.lower() == "id3":
            impurity_func = self.entropy
        elif self.criterion.lower() == "gini":
            impurity_func = self.gini
        
        best_gain = -np.inf
        best_feature = None
        best_threshold = None
        
        for feature_idx in range(n_features):
            # valores ordenados de la característica
            x_column = X[:, feature_idx]
            # valores candidatos (puntos medios entre valores únicos)
            unique_vals = np.unique(x_column)
            if len(unique_vals) == 1:
                continue  # no se puede dividir si todo es igual
            
            # candidatos tipo sklearn: puntos medios
            thresholds = (unique_vals[:-1] + unique_vals[1:]) / 2.0
            
            for threshold in thresholds:
                left_mask = x_column <= threshold
                right_mask = ~left_mask
                
                y_left = y[left_mask]
                y_right = y[right_mask]
                
                gain = self.information_gain(y, y_left, y_right, impurity_func)
                
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature_idx
                    best_threshold = threshold
        
        if best_gain <= 0:
            return None, None
        
        return best_feature, best_threshold

    # Construcción del árbol
    def build_tree(self, X, y, depth):
        # Nodo hoja si: puro, max_depth alcanzada o muy pocas muestras
        num_samples = len(y)
        unique_classes, counts = np.unique(y, return_counts=True)
        majority_class = unique_classes[np.argmax(counts)]
        
        node = {
            "is_leaf": False,
            "prediction": majority_class,
            "feature_index": None,
            "threshold": None,
            "left": None,
            "right": None
        }
        
        # criterios de parada
        if len(unique_classes) == 1:  # todas las etiquetas iguales
            node["is_leaf"] = True
            return node
        
        if self.max_depth is not None and depth >= self.max_depth:
            node["is_leaf"] = True
            return node
        
        if num_samples < self.min_samples_split:
            node["is_leaf"] = True
            return node
        
        # buscar mejor split
        feature_idx, threshold = self.best_split(X, y)
        
        if feature_idx is None:  # no mejora
            node["is_leaf"] = True
            return node
        
        # crear hijos
        left_mask = X[:, feature_idx] <= threshold
        right_mask = ~left_mask
        
        X_left, y_left = X[left_mask], y[left_mask]
        X_right, y_right = X[right_mask], y[right_mask]
        
        node["feature_index"] = feature_idx
        node["threshold"] = threshold
        node["left"] = self.build_tree(X_left, y_left, depth + 1)
        node["right"] = self.build_tree(X_right, y_right, depth + 1)
        
        return node

    # Funciones fit() y predict()
    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)
        self.tree_ = self.build_tree(X, y, depth=0)
        return self
    
    def predict_one(self, x, node):
        if node["is_leaf"]:
            return node["prediction"]
        
        if x[node["feature_index"]] <= node["threshold"]:
            return self.predict_one(x, node["left"])
        else:
            return self.predict_one(x, node["right"])
    
    def predict(self, X):
        X = np.asarray(X)
        return np.array([self.predict_one(x, self.tree_) for x in X])

In [ ]:
df = pd.read_csv("data/data_clasificacion.csv")

In [ ]:
X = df.drop(columns=["Class"]).values
y = df["Class"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)



model = myDecisionTreeClassifier(
    criterio="gini",
    max_depth=10,
    min_samples_split=2
)

model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 weighted:", f1_score(y_test, y_pred, average='weighted'))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))



Accuracy: 0.4505445654589909
F1 weighted: 0.4194611092861488
Matriz de confusión:
 [[ 93   0   0  13  19   0   0  14   0  16   1]
 [  0   8   4   0   0  17  48   0   8  49 209]
 [  0   2  57   0   0  11  31   0   2  34 181]
 [ 21   0   0  57   4   0   0   8   0   7   3]
 [ 18   0   0   1  44   0   0   1   0  29   4]
 [  0   3   8   0   0 222  19   0   0  72  38]
 [  0  14  10   0   0  35  97   0  30 115 346]
 [ 19   0   0  13   2   0   0 110   0   0   0]
 [  0   5   1   0   0   3  10   0 230   2 213]
 [ 13   5  12   2  17  51  42   3   4 264 218]
 [ 11  10  26   6   5  21  89   2 126  96 845]]
